In [36]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

sns.set()

In [38]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

In [40]:
assets = ['MSFT', 'AAPL', 'META', 'SPY']
models = {}
seq_len = 10
for asset in assets:  
    df = pd.read_csv(asset+'.csv')
#     if 'GOOG' not in asset:
#         df['Date'] = df['Date'].astype(str)
#         df['Date'] = pd.to_datetime(df['Date'],format='%m/%d/%Y')
#         df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    #df.set_index('Date',inplace=True)
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']])
    
    train_data, test_data = train_test_split(df_scaled, test_size=0.3, shuffle=True, random_state=13)
    
    # Create sequences for training and testing
    x_train, y_train = create_sequences(train_data, seq_len)
    x_test, y_test = create_sequences(test_data, seq_len)
    
    models[asset] = df
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(seq_len,6), return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(6))
    
    for layer in model.layers[:-1]:
        layer.trainable = False
    
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    model.fit(x_train,y_train, epochs=10,batch_size=32, validation_data=(x_test, y_test))
    
    models[asset] = {'model':model,'scaler':scaler}

x_test_META, y_test_META = models['META']['x_test'],models['META']['y_test']
predictions_META = models['META'].predict(x_test_META)

predictions_actual_META = models['META']['scaler'].inverse_transform(predictions_META)


Epoch 1/10

94/94 [==============================] - 2s 8ms/step - loss: 0.0745 - val_loss: 0.0569
Epoch 2/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0592 - val_loss: 0.0515
Epoch 3/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0560 - val_loss: 0.0510
Epoch 4/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0555 - val_loss: 0.0510
Epoch 5/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0555 - val_loss: 0.0510
Epoch 6/10
94/94 [==============================] - 0s 5ms/step - loss: 0.0554 - val_loss: 0.0511
Epoch 7/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0554 - val_loss: 0.0510
Epoch 8/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0554 - val_loss: 0.0511
Epoch 9/10
94/94 [==============================] - 0s 5ms/step - loss: 0.0554 - val_loss: 0.0510
Epoch 10/10
94/94 [==============================] - 0s 4ms/step - loss: 0.0554 - val_loss: 0.0511
Epoch 1/10
94/94 

KeyError: 'x_test'

In [42]:
print(models)

{'MSFT': {'model': <keras.src.engine.sequential.Sequential object at 0x0000024BD435F250>, 'scaler': MinMaxScaler()}, 'AAPL': {'model': <keras.src.engine.sequential.Sequential object at 0x0000024BD4ACDB50>, 'scaler': MinMaxScaler()}, 'META': {'model': <keras.src.engine.sequential.Sequential object at 0x0000024BD7911730>, 'scaler': MinMaxScaler()}, 'SPY': {'model': <keras.src.engine.sequential.Sequential object at 0x0000024BDA63F640>, 'scaler': MinMaxScaler()}}


In [16]:
train_assets = ['MSFT','AAPL','META']
test_assets = ['SPY','GOOG']

train_data = df[train_assets]
test_data = df[test_assets]

scaler = MinMaxScaler(feature_range=(0,1))
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

KeyError: "None of [Index(['MSFT', 'AAPL', 'META'], dtype='object')] are in the [columns]"